In [1]:
from urllib.request import Request, urlopen
from bs4 import BeautifulSoup
import requests
from datetime import datetime as dt
from datetime import timedelta
from pytz import timezone
from time import time, sleep


#Pass the URL Of the Science Daily Website to Update the Database of the Educracypedia 
def insert(l):
    url = l
    req = Request(url, headers={'User-Agent': 'Mozilla/5.0'})
    webpage = urlopen(req,timeout=10).read()
    from datetime import datetime

    # current date and time
    now = datetime.now()

    timestamp = datetime.timestamp(now)
    print("timestamp =", timestamp)
    dt_object = datetime.fromtimestamp(timestamp)

    print("dt_object =", dt_object)


    soup = BeautifulSoup(webpage)
    para = " "
    term = " "
    title = soup.find(id="headline").string
    slug = title.replace(" ", "-").lower()
    subtitle = soup.find(id='subtitle')
    paragraph = soup.find(id="story_text").find_all('p')[:-4]
    for a in paragraph:
        para = para + str(a)
    source =  soup.find(id="source").string
    summary = soup.find(id="abstract").string
    journal = soup.find(id="journal_references").find('li')
    terms = soup.find(id="related_terms").find_all('a')
    for a in terms:
        term = term + a.string + ","
    MainCat = soup.find(id="related_topics").find_all('a')[0]
    Cat = soup.find(id="related_topics").find_all('a')[1]
    query = """INSERT INTO news_news ( title, slug, updated_on, content, created_on, image, status, source, summary, author_id, category_id, journal_reference) VALUES (%s, %s, %s ,%s, %s ,'uploads/news/logo.jpg',1, %s, %s,1, 27, %s ) """
    print("Title" + title)
    print("Tags " + term)
    print("Main Category  " + str(MainCat.string))
    print("Category " + str(Cat.string))

    import mysql.connector
    from mysql.connector import Error

    try:
        connection = mysql.connector.connect(host='database-1.cez5x0venctn.ap-south-1.rds.amazonaws.com',
                                             database='MAYUR',
                                             user='admin',
                                             password='qwertyuiop')
        if connection.is_connected():
            db_Info = connection.get_server_info()
            print("Connected to MySQL Server version ", db_Info)
            cursor = connection.cursor(prepared=True)
            cursor.execute("select database();")
            record = cursor.fetchone()
            print("You're connected to database: ", record)
            tuple1 = (title, slug, dt_object, str(para),dt_object, source, summary, str(journal))
            cursor.execute(query, tuple1)
            connection.commit()
            print(cursor.rowcount, "Record inserted successfully into news table")

    except Error as e:
        print("Error while connecting to MySQL", e)
        


In [3]:
insert("https://www.sciencedaily.com/releases/2022/02/220225085906.htm")

timestamp = 1645945716.493087
dt_object = 2022-02-27 12:38:36.493087
TitleStudy questions the role of vitamin D2 in human health but its sibling, vitamin D3, could be important for fighting infections
Tags  B vitamins,Vitamin A,Vitamin E,Vitamin D,Pernicious anemia,Rickets,Vegetable,Shiitake mushroom,
Main Category  Health & Medicine
Category Vitamin D
Connected to MySQL Server version  8.0.27
You're connected to database:  ('MAYUR',)
1 Record inserted successfully into news table
